In [18]:
import pandas as pd
import os
import re
import datetime as dt
import numpy as np
import csv

In [19]:
fillValue = '-9999999'

In [20]:
compileDict = {}

In [21]:
discreteSummaryFile = 'Cabled-11_TN382_Discrete_Summary_2021-05-21_ver_1.00.csv'
READMEfile = 'Cabled-11_README_notes'

dataPath = '/Users/rsn/Desktop/FileCabinet/Cruises/Cabled-11_TN382'

CTDfilePath = 'Alfresco/CTD Data'
ROVfilePath = 'Alfresco/ROV Data'
compileFile = 'TN382_DiscreteCastLogs_v2.xlsm'
bottleParams = '/Users/rsn/Desktop/FileCabinet/DiscreteSampling/DiscreteSummaries/bottleMap.csv'
headerFile = '/Users/rsn/Desktop/FileCabinet/DiscreteSampling/DiscreteSummaries/discreteSummaryHeaderMap.csv'
nutFile = 'Cabled-11_TN382_Nutrients_Sample_Data_2021-04-29_ver-1.01.xlsx'
fluorFile = 'Cabled-11_TN382_Chlorophyll_Sample_Data_2021-04-29.xlsx'
salFile = 'Cabled-11_TN382_Salinity_Sample_Data_2021-04-29_ver_1-01.xlsm'
flagFile = 'flags.csv'
flagMapFile = 'flagMap.csv'

In [22]:
### Load in map for discrete summary headers to data dictionaries created by loading excel sheets
headerMap_dict = pd.read_csv(headerFile, index_col=0, squeeze=True).to_dict()

### Load in map to assign column headers in bottle file to discrete summary columns
bottleMap_dict = pd.read_csv(bottleParams, index_col=0, squeeze=True).to_dict()

### Load in map to assign flag header columns to flag type
flagHeadersMap_dict = pd.read_csv(flagMapFile, index_col=0, squeeze=True).to_dict()

### Load in map to assign flag type to flag string and bit position
flagBitMap_dict = pd.read_csv(flagFile, index_col=0, squeeze=True).to_dict()

In [23]:
def flagBits(flagStrings,column):
    bits = []
    flagBitMap = list('*0000000000000000')
    flagKey = flagHeadersMap_dict[column]
    flagColumns = flagBitMap_dict[flagKey]
    strings = flagStrings.split(',')
    for flagStr in strings:
        for k,v in flagColumns.items():
            if flagStr.strip() in v:
                bits.append(k)

    for bit in bits:
        bitPosition = 16 - int(bit)
        flagBitMap[bitPosition] = '1'
    flag = "".join(flagBitMap)
    return flag

In [24]:
def parseBottleFile(btlFile):
    castDict = {}
    
    f = open(btlFile, 'r')
    btlLines = f.readlines()
    bottleTimes = []
    bottleData = []
    for line in btlLines:
        if line.startswith('*') or line.startswith('#') or re.search(r'.*Bottle.*Date.*',line) or re.search(r'.*Position.*Time.*',line):
            if '* NMEA Latitude' in line:
                # * NMEA Latitude = 45 49.81 N
                m = re.search(r'.* NMEA Latitude\s=\s(\d*)\s(\d*.\d*)\s.*',line)
                if m:
                    latitude = float(m.group(2))/60 + float(m.group(1))
                    castDict['Start Latitude [degrees]'] = latitude
            if '* NMEA Longitude' in line:
                # * NMEA Longitude = 129 44.77 W
                m = re.search(r'.* NMEA Longitude\s=\s(\d*)\s(\d*.\d*)\s.*',line)
                if m:
                    long = float(m.group(2))/60 + float(m.group(1))
                    # make longituede negative
                    castDict['Start Longitude [degrees]'] = -long
            if '* NMEA UTC' in line:
                # * NMEA UTC (Time) = Jul 30 2017 11:01:22
                # * NMEA UTC (Time) = Aug 03 2020  15:17:41
                # convert to: 2017-07-30T11:01:22.000Z
                m = re.search(r'.*NMEA\sUTC.*=.*([a-zA-Z]{3}).*(\d{2}).*(\d{4}).*(\d{2}:\d{2}:\d{2}).*',line)
                if m:
                    monthInt = '%02d' % dt.datetime.strptime(m.group(1), "%b").month
                    timeString = m.group(3) + '-' + str(monthInt) + '-' + m.group(2) + 'T' + m.group(4) + '.000Z'
                    castDict['Start Time [UTC]'] = timeString
            if re.search(r'.*Bottle.*Date.*',line):
                bottleHeader = line.split()
        else:
            dataLines = line.split()
            if re.search(r'.*\d{2}:\d{2}:\d{2}.*',dataLines[0]):
                bottleTimes.append(dataLines[0])
            elif re.search(r'.*[1-9]|1[1-9]|2[1-4].*',dataLines[0]):
                bottleData.append(dataLines)
    bottleHeader = bottleHeader[2:]
    castDict.setdefault('BottleData',{})
    for i in range(len(bottleData)):
        bottle = int(bottleData[i][0])
        castDict['BottleData'].setdefault(bottle,{})
        monthInt = '%02d' % dt.datetime.strptime(bottleData[i][1], "%b").month
        timeString = bottleData[i][3] + '-' + str(monthInt) + '-' + bottleData[i][2] + 'T' + bottleTimes[i] + '.000Z'
        castDict['BottleData'][bottle]['CTD Bottle Closure Time [UTC]'] = timeString
        dataList = bottleData[i][4:-1]
        for j in range(len(bottleHeader)):
            if bottleHeader[j] in bottleMap_dict:
                castDict['BottleData'][bottle][bottleMap_dict[bottleHeader[j]]] = dataList[j]

    return castDict

In [25]:
def parseROVfile(ROVfile):
    ROVdata = []
    
    f = open(ROVfile, 'r')
    ROVlines = f.readlines()
    bottleTimes = []
    bottleData = []
    for line in ROVlines:
        if line.startswith('CT2'):
            dataLine = line.split()
            date = dataLine[1].split('/')
            timeString = date[0] + '-' + date[1] + '-' + date[2] + 'T' + dataLine[2] + 'Z'
            #print(timeString)
            #4 = temp,5 = cond,6=press,8=sal
            ROVdata.append([timeString,float(dataLine[6].strip(',')),float(dataLine[4].strip(',')),float(dataLine[5].strip(',')),float(dataLine[8].strip(','))])

    df_ROV = pd.DataFrame(ROVdata, columns = ['ROV Time', 'CTD Pressure [db]','CTD Temperature 1 [deg C]', 'CTD Conductivity 1 [S/m]','CTD Salinity 1 [psu]'])

    return df_ROV

In [26]:
def meanROVdata(sampleTime,meanWindow_mins,df_ROV):
    bottleTime = dt.datetime.strptime(sampleTime,"%Y-%m-%dT%H:%M:%S.%fZ")
    windowStart = bottleTime - dt.timedelta(minutes=meanWindow_mins)
    df_ROV['ROV Time'] = pd.to_datetime(df_ROV['ROV Time'], format='%Y-%m-%dT%H:%M:%S.%fZ')
    extractMask = (df_ROV['ROV Time'] > windowStart) & (df_ROV['ROV Time'] <= bottleTime)
    df_ROV = df_ROV.loc[extractMask].mean()
    
    return df_ROV

In [27]:
ROVfileList = []
ROVdict = {}

for rootdir, dirs, files in os.walk(os.path.join(dataPath, ROVfilePath)):
    for file in files:
        if ('.CT2' in file):
            ROVfile_full = os.path.join(rootdir, file)
            ROVfileList.append(ROVfile_full)

for ROVfile in ROVfileList:
    m = re.search(r'.*/([A-Z0-9]*_J2-\d*_\d*_\d*.*).CT2', ROVfile)
    if m:
        cast = m.group(1)
        ROVdict[cast] = parseROVfile(ROVfile)            
    else:
        print('error retrieving cast number from file: ', ROVfile)
        
#print(ROVdict)
compileDict['ROVdict'] = ROVdict

In [28]:
bottleFileList = []
bottleDict = {}

for rootdir, dirs, files in os.walk(os.path.join(dataPath, CTDfilePath)):
    for file in files:
        if ('.btl' in file):
            btlFile_full = os.path.join(rootdir, file)
            bottleFileList.append(btlFile_full)

for btlFile in bottleFileList:
    m = re.search(r'.*(CTD-\d*).btl', btlFile)
    if m:
        cast = m.group(1)
        bottleDict[cast] = parseBottleFile(btlFile)            
    else:
        print('error retrieving cast number from file: ', btlFile)
        
#print(bottleDict)
compileDict['bottleDict'] = bottleDict

In [29]:
# Import sheets from discrete summary compilation spreadsheet
df_casts = pd.read_excel(os.path.join(dataPath, compileFile),sheet_name = 'CastList',engine='openpyxl')

df_samples = pd.read_excel(os.path.join(dataPath, compileFile),sheet_name = 'SampleList',engine='openpyxl')
df_samples = df_samples[~df_samples['Cast'].isnull()]
df_samples.fillna(fillValue, inplace = True)

df_oxygen = pd.read_excel(os.path.join(dataPath, compileFile),sheet_name = 'OxygenLog_all',engine='openpyxl')
df_oxygen = df_oxygen[~df_oxygen['Unnamed: 3'].isnull()].reset_index()
new_header = df_oxygen.iloc[0] #grab the first row for the header
df_oxygen.fillna(fillValue, inplace = True)
df_oxygen = df_oxygen[1:].astype(str) #take the data less the header row, convert all to strings
df_oxygen.columns = new_header #set the header row as the df header
df_oxygen.rename(columns={'Oxygen Concentration (mL/L)':'Discrete Oxygen [mL/L]'}, inplace=True)
compileDict['df_oxygen'] = df_oxygen.filter(['Sample Bottle #', 'Discrete Oxygen [mL/L]'])

df_chlorophyll = pd.read_excel(os.path.join(dataPath, compileFile),sheet_name = 'ChloroLog_all',engine='openpyxl')
df_chlorophyll = df_chlorophyll[~df_chlorophyll['Unnamed: 3'].isnull()].reset_index()
new_header = df_chlorophyll.iloc[0] #grab the first row for the header
df_chlorophyll.fillna(fillValue, inplace = True)
df_chlorophyll = df_chlorophyll[1:].astype(str) #take the data less the header row
df_chlorophyll.columns = new_header #set the header row as the df header
compileDict['df_chlorophyll'] = df_chlorophyll.filter(['Sample Bottle #', 'Chlorophyll Vial'])

compileDict['df_CastLog_ROV'] = pd.read_excel(os.path.join(dataPath, compileFile),sheet_name = 'CastLog_ROV',engine='openpyxl')
compileDict['df_CTDflags'] = pd.read_excel(os.path.join(dataPath, compileFile),sheet_name = 'CTDflags',engine='openpyxl')
compileDict['df_CTDflags_ROV'] = pd.read_excel(os.path.join(dataPath, compileFile),sheet_name = 'CTDflags_ROV',engine='openpyxl')

In [30]:
# Create metadata dictionary from castList
metadataDict = {}
df_casts_columns = df_casts.columns.to_list()
df_casts_columns.remove('Cast')
df_casts.fillna(fillValue, inplace = True)
for index, row in df_casts.iterrows():
    if not isinstance(row['Cruise'], float):
        metadataDict.setdefault(row['Cast'],{})
        for col in df_casts_columns:
            metadataDict[row['Cast']][col] = row[col]
            
compileDict['metadataDict'] = metadataDict

In [31]:
# Load sample data from outside labs

### Nutrients ###
nutData = pd.read_excel(os.path.join(dataPath, nutFile),sheet_name = None, engine='openpyxl')

df_list = []
nut_header = ['index','bottle #','Discrete Phosphate [uM]','Discrete Silicate [uM]','Discrete Nitrate [uM]','Discrete Nitrite [uM]','Discrete Ammonium [uM]']
for k,v in nutData.items():
    filteredData = v[ (~v['Unnamed: 1'].isnull()) & (v['Unnamed: 2'].str.contains('CTD','J2-').any())][['Unnamed: 1','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10']].reset_index()
    #filteredData = v[ (~v['Unnamed: 1'].isnull()) & 
    #                 (any(castStr in v['Unnamed: 2'] for castStr in ['CTD','J2-']))][['Unnamed: 1','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10']].reset_index()
    filteredData.columns = nut_header
    df_list.append(filteredData)
    
compileDict['df_nuts'] = pd.concat(df_list, axis=0).fillna(fillValue)

### Salinity ###
salData = pd.read_excel(os.path.join(dataPath, salFile),sheet_name = None, engine='openpyxl')
df_list = []
sal_header = ['index','bottle #','Discrete Salinity [psu]']
for k,v in salData.items():
    filteredData = v[~v['Unnamed: 4'].isnull()][['Unnamed: 4','Unnamed: 6']].reset_index()
    #filteredData = v[(~v['Unnamed: 4'].isnull()) & (v['Unnamed: 4'].str.isnumeric())][['Unnamed: 4','Unnamed: 6']].reset_index()
    filteredData.columns = sal_header
    df_list.append(filteredData)
    
compileDict['df_sal'] = pd.concat(df_list, axis=0).fillna(fillValue)
    
### Chlorophyll ###
fluorData = pd.read_excel(os.path.join(dataPath, fluorFile),sheet_name = None, engine='openpyxl')
df_list = []
fluor_header = ['index','bottle #','Discrete Chlorophyll [ug/L]','Discrete Phaeopigment [ug/L]','Discrete Fo/Fa Ratio']
for k,v in fluorData.items():
    filteredData = v[(~v['Unnamed: 4'].isnull()) & (v['Unnamed: 4'].apply(isinstance, args=(int,)))][['Unnamed: 4','Unnamed: 10','Unnamed: 11','Unnamed: 12']].reset_index()
    filteredData.columns = fluor_header
    df_list.append(filteredData)
    
compileDict['df_fluor'] = pd.concat(df_list, axis=0).fillna(fillValue)


In [32]:
headers = [k for k,v in headerMap_dict.items()]
dataRows = []
for index, row in df_samples.iterrows():
    dataRow = []
    for column in headers:
        source = headerMap_dict[column]
        if ',' in source:
            sources = source.split(',')
            if 'CTD' in row.Cast:
                source = sources[0]
            elif 'J2' in row.Cast:
                source = sources[1]      
        if 'df_samples' in source:
            if 'Flag' in column:
                if fillValue in row[column]:
                    dataCell = fillValue
                else:
                    flagStrings = row[column]
                    dataCell = flagBits(flagStrings, column)
            else:
                dataCell = row[column]
            
        elif any(sourceString in source for sourceString in ['metadataDict','bottleDict']):
            if 'bottleDict_bottle' in source:
                dataCell = compileDict['bottleDict'][row.Cast]['BottleData'][row['Niskin/Bottle Position']][column]  
            else:
                if 'Flag' in column:
                    flagStrings = compileDict[source][row.Cast][column]
                    if fillValue in flagStrings:
                        dataCell = fillValue
                    else:
                        dataCell = flagBits(flagStrings, column)
                else:
                    dataCell = compileDict[source][row.Cast][column]
            
        elif any(sourceString in source for sourceString in ['df_CTDflags','df_CTDflags_ROV']):
            df = compileDict[source]
            cell = df.loc[df['Parameter'] == column, 'Parameter Flag']
            if len(cell) > 0:
                flagStrings = cell.values[0]
                dataCell = flagBits(flagStrings,column)
            else:
                dataCell = fillValue
                
        elif 'df_CastLog_ROV' in source:
            df = compileDict[source]
            cell = df.loc[((df['Dive'] == row.Cast) & (df['Niskin'] == row['Niskin/Bottle Position'])), column]
            dataCell = cell.values[0]

        elif 'ROVdict' in source:
            df = compileDict['df_CastLog_ROV']
            bottleTime = df.loc[((df['Dive'] == row.Cast) & (df['Niskin'] == row['Niskin/Bottle Position'])), 'CTD Bottle Closure Time [UTC]'].values[0]
            ROVfile = compileDict['metadataDict'][row.Cast]['CTD File']
            cell = meanROVdata(bottleTime,2,ROVdict[ROVfile])
            dataCell = cell[column]
            
        elif 'fill' in source:
            dataCell = fillValue

        elif 'df_oxygen' in source:
            if row['Oxygen Bottle Number'] != fillValue:
                df = compileDict[source]
                cell = df.loc[df['Sample Bottle #'] == str(row['Oxygen Bottle Number']) , column]
                if len(cell) > 0:
                    dataCell = cell.values[0]
                else:
                    print('error retrieving oxygen values...')
            else:
                dataCell = fillValue

        elif 'df_fluor' in source:
            if row['Chlorophyll Bottle Number'] != fillValue:
                df_chloro = compileDict['df_chlorophyll']
                vialNumber = df_chloro.loc[df_chloro['Sample Bottle #'] == str(int(row['Chlorophyll Bottle Number'])),  'Chlorophyll Vial'].values[0]
                df = compileDict[source]
                df['bottle #'] = df['bottle #'].astype(str)
                cell = df.loc[df['bottle #'] == vialNumber, column]
                if len(cell) > 0:
                    dataCell = cell.values[0]
                else:
                    print('error retrieving chlorphyll values...')
            else:
                dataCell = fillValue
       
        elif 'df_nuts' in source:
            if row['Nutrient Bottle Number' ] != fillValue:
                df = compileDict[source]
                df['bottle #'] = df['bottle #'].astype(str)
                cell = df.loc[df['bottle #'] == str(row['Nutrient Bottle Number']) , column]
                if len(cell) > 0:
                    dataCell = cell.values[0]
                else:
                    print('error retriving nutrient values...')
                    print(str(row['Nutrient Bottle Number']))
                    print(df['bottle #'])
            else:
                dataCell = fillValue
            
        elif 'df_sal' in source:
            if row['Salinity Bottle Number'] != fillValue:
                df = compileDict[source]
                df['bottle #'] = df['bottle #'].astype(str)
                cell = df.loc[df['bottle #'] == row['Salinity Bottle Number'], column]
                if len(cell) > 0:
                    dataCell = cell.values[0]
                else:
                    print('error retrieving salinity values...')
            else:
                dataCell = fillValue
            
        elif 'df_DIC' in source:
            if row['DIC Bottle Number'] != fillValue:
                df = compileDict[source]
                cell = df.loc[df['bottle #'] == row['DIC Bottle Number'] , column]
                if len(cell) > 0:
                    dataCell = cell.values[0]
                else:
                    print('error retrieving carbon values...')     
            else:
                dataCell = fillValue
                
                
        dataRow.append(dataCell)
    dataRows.append(dataRow)
        

In [33]:
# append CTD parameter flags if discrete sample is present
# discrete samples for salinity, oxygen, chlorophyll, pH

discreteCTDmatch = {'Discrete Salinity [psu]':['CTD Conductivity 1 Flag','CTD Conductivity 2 Flag'],
                   'Discrete Oxygen [mL/L]':['CTD Oxygen Flag'],
                   'Discrete Chlorophyll [ug/L]':['CTD Fluorescence Flag'],
                   'Discrete pH [Total scale]':['CTD pH Flag'],
                   'Calculated pH':['CTD pH Flag']}

for row in dataRows:
    for key in discreteCTDmatch:
        discreteIndex = headers.index(key)
        if fillValue not in str(row[discreteIndex]):
            for flag in discreteCTDmatch[key]:
                flagIndex = headers.index(flag)
                if fillValue not in row[flagIndex]:
                    newFlag = list(row[flagIndex])
                    newFlag[9] = '1'
                    newFlagString = "".join(newFlag)
                    row[flagIndex] = newFlagString
    

In [34]:
with open(discreteSummaryFile, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(dataRows)

In [ ]:
### Output list of CTD files for re-naming list in README
ctdFiles = []
for key, value in compileDict['metadataDict'].items():
    if fillValue not in key:
        ctdFiles.append(compileDict['metadataDict'][key]['CTD File'])
        

In [ ]:
### Output all Notes from CastList and SampleList for README

notes = []
for index, row in df_samples.iterrows():
    if fillValue not in row.Notes:
        cruise = compileDict['metadataDict'][row.Cast]['Cruise']
        noteLine = cruise + ', ' + row.Cast + ', ' + row.Notes
        notes.append(noteLine)

for key,value in compileDict['metadataDict'].items():
    if fillValue not in compileDict['metadataDict'][key]['Notes']:
        cruise = compileDict['metadataDict'][key]['Cruise']
        note = compileDict['metadataDict'][key]['Notes']
        noteLine = cruise + ', ' + key + ', ' + note
        notes.append(noteLine)


In [ ]:
f = open(READMEfile, "w")
f.write('File Mapping:\n')
for line in ctdFiles:
    f.write(line + "\n")

f.write('Summary Notes:\n')
for line in notes:
    f.write(line + "\n")

f.close()